In [1]:
import glob
import os
import numpy as np
import math
from DataProcess import *
from SolutionChecker import *
from LocalSearch import *

# Test for Input TD

In [2]:
# data_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.data"

In [3]:
# solution_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.solution"

In [4]:
# print_data(data_path)

In [5]:
#print_solution(solution_path)

In [6]:
#verify_solution(data_path,solution_path)

In [7]:
# EVA_TREE,GRAPH,NB_NODES = read_data(data_path)
# NB_EVA_NODES = len(EVA_TREE)
# NB_EDGES = len(GRAPH)
# LIST_EVA_NODES = [item[0] for item in EVA_TREE]

In [8]:
# for i in range(NB_EVA_NODES) : 
#     node_id = EVA_TREE[i][0]
#     print('Evacuation of (only) node ',node_id,' is ', get_duration(node_id,EVA_TREE,GRAPH),' mins')

In [9]:
# print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

In [10]:
# endtime,init_sol= get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)
# print('Borne superieur = {} mins'.format(endtime))
# init_sol

In [11]:
# cp = cProfile.Profile()
# cp.enable()
# endtime,best_solution= LocalSearchRun2(init_sol,EVA_TREE,GRAPH,n_iter=10)
# cp.disable()
# cp.print_stats()

# Test for given instances 

In [12]:
# data_path_2 = os.path.dirname(os.path.abspath('__file__')) + "/InstancesInt/dense_10_30_3_1_I.full"
# EVA_TREE,GRAPH,NB_NODES = read_data(data_path_2)
# NB_EVA_NODES = len(EVA_TREE)
# NB_EDGES = len(GRAPH)
# LIST_EVA_NODES = [item[0] for item in EVA_TREE]

In [13]:
#print_data(data_path_2)

In [14]:
# print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

In [15]:
# endtime,init_sol= get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)
# print('Borne superieur = {} mins'.format(endtime))
# init_sol

In [16]:
# cp = cProfile.Profile()
# cp.enable()
# endtime,best_solution= LocalSearchRun(init_sol,EVA_TREE,GRAPH,n_iter=10)
# cp.disable()
# cp.print_stats()

In [17]:
# cp = cProfile.Profile()
# cp.enable()
# endtime,best_solution= LocalSearchRun4(init_sol,EVA_TREE,GRAPH,n_iter=10)
# cp.disable()
# cp.print_stats()

In [18]:
# cp = cProfile.Profile()
# cp.enable()
# best_solution,endtime = LocalSearchRun2(init_sol,EVA_TREE,GRAPH,n_iter=10)
# cp.disable()
# cp.print_stats()

In [19]:
# print('solution = ',best_solution)
# print('end time = ',endtime)

In [20]:
# cp = cProfile.Profile()
# cp.enable()
# endtime,best_solution,algo,exc = LocalSearchRandomStart2(EVA_TREE,GRAPH,n_iter=10,n_start_points=5)
# create_solution_file3('dense_10_30_3_1_I',best_solution,endtime,algo,exc)
# cp.disable()
# cp.print_stats()
# print('solution = ',best_solution)
# print('end time = ',endtime)

In [21]:
# create_solution_file('data/wildfire_sup.solution',best_solution,endtime)

# Test with due_date problem

In [2]:
import glob
import os
import numpy as np
import math
from DataProcess import *
from SolutionChecker import *
from LocalSearch import *

In [3]:
data_path = os.path.dirname(os.path.abspath('__file__')) + "/InstancesInt/sparse_10_30_3_1_I.full"
# data_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.data"
EVA_TREE,GRAPH,NB_NODES = read_data(data_path)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

In [88]:

# #Run a local search to find a solution
# endtime,best_solution,algo,exc = LocalSearchRandomStart(EVA_TREE,GRAPH,n_iter=10,n_start_points=1)
# create_solution_file('sparse_10_30_3_3_I',best_solution,endtime,algo,exc)

# print('solution = ',best_solution)
# print('end time = ',endtime)

# # verify this solution by our checker
# solution_path = os.path.dirname(os.path.abspath('__file__')) + "/Solutions/sparse_10_30_3_1_I.solution"
# verify_solution(data_path,solution_path)


In [17]:
def get_end_time_beta(list_eva_nodes,eva_tree,graph) : 
    ## Initialize the ressources 
    ressources = {}
    for edge in graph :
        edge_cap = edge[-1]
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(300,edge_cap))   
        
    ## Arrange the tasks 
    tasks = {}
    for i in list_eva_nodes : 
        cap_ok = False
        shift_time = 0
        dist = 0
        while(not cap_ok) :
            start = shift_time

            ## arrange the following tasks with the start
            duration,demande_res,eva_rate = get_task(i,eva_tree,graph)
            current = i
            
            for j in demande_res : 
                nxt = j
                if current != nxt :
                    due_date,length,edge_cap = get_edge_info(current,nxt,graph)
                        
                    if current < nxt :
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                    else : 
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(nxt,current)]))
                    
                    dispo[start:start+duration] -= eva_rate
                    if shift_time > 0 :
                        dispo[start-dist:start-dist+duration] += eva_rate
#                         print(start-shift_time,start-shift_time+duration)
#                     print('Cap of edge[{}-{}] = {}'.format(current,nxt,dispo))
                    ## Update the ressources of the current edge
                    if current < nxt :
                        ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo
                    else : 
                        ressources['Cap of edge[{}-{}]'.format(nxt,current)] = dispo
                    
                    
#                     if shift_time == 0 :
#                         tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,eva_rate,due_date,dispo])
#                     else :
                    tasks['Evacuees from {} at edge [{}-{}]'.format(i,current,nxt)] = [start,start+length+duration,duration,eva_rate,due_date,dispo]
                    
                    start += length
                current = j
                
            ## check the capacity constraints with this time start
            stop = False
            
            for key in tasks :
                if not stop :
                    check_dispo = [item for item in tasks[key][5] if item<0]
                    if len(check_dispo) > 0 :
#                         print(key,tasks[key][5])
                        shift_time += len(check_dispo)
#                         print(dist)
                        dist = len(check_dispo)
                        stop = True
                        cap_ok = False
                    else :
                        cap_ok = True
                ## return cap_ok and shift_time 



                
#         print('After node',i,' tasks = ',tasks)
#     print('tasks = ', tasks)
#     print('Nb of tasks = ',len(tasks))
    end_time = np.max([tasks[keys][1] for keys in tasks])
#     print([tasks[keys][1] for keys in tasks])
#    print('End time is : ',end_time)
    solution = create_solution(tasks,LIST_EVA_NODES)
    return end_time,solution

random.shuffle(LIST_EVA_NODES)
end,sol = get_end_time(LIST_EVA_NODES,EVA_TREE,GRAPH)
print(end,sol)
print(check_a_solution(data_path,sol))
end,sol = get_end_time_beta(LIST_EVA_NODES,EVA_TREE,GRAPH)
print(end,sol)
print(check_a_solution(data_path,sol))


159 [[128, 134, 0], [77, 206, 14], [190, 74, 1], [107, 72, 31], [130, 206, 78], [138, 206, 93], [134, 72, 82], [177, 71, 92], [156, 72, 0], [167, 72, 59]]
CREATION TASKS & VERIFY CONSTRAINTS
-----------------------------
CHECK F_OBJECTIF
-----------------------------
End time is :  159
-----------------------------
END
-----------------------------
True
159 [[128, 134, 0], [77, 206, 14], [190, 74, 1], [107, 72, 31], [130, 206, 78], [138, 206, 93], [134, 72, 82], [177, 71, 92], [156, 72, 0], [167, 72, 0]]
CREATION TASKS & VERIFY CONSTRAINTS
-----------------------------
CHECK F_OBJECTIF
-----------------------------
End time is :  159
-----------------------------
END
-----------------------------
True


In [3]:

def get_end_time_alpha(list_eva_nodes,eva_tree,graph) : 
    ## Initialize the ressources 
    ressources = {}
    for edge in graph :
        edge_cap = edge[-1]
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(1000,edge_cap))   
        
    ## Arrange the tasks 
    tasks = {}
    for i in list_eva_nodes : 
        start = 0
        duration,demande_res,eva_rate = get_task(i,eva_tree,graph)
        current = i
        for j in demande_res : 
            nxt = j
            if current != nxt :
                due_date,length,edge_cap = get_edge_info(current,nxt,graph)
                ## Check the capacity constraints
                ok = False
                cumul_time = 0
                while (not ok) :
                    if current < nxt :
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                    else : 
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(nxt,current)]))
                    dispo[start:start+duration] -= eva_rate
                    check_dispo = [item for item in dispo if item < 0]
                    if (len(check_dispo) > 0) :   
                        ## Shift the task if overload
                        cumul_time += len(check_dispo) 
                        ok = False
                        ## Change the time start of the previous nodes 
                        for keys in tasks :
                            if 'Evacuees from {}'.format(i) in keys :
                                
                                tasks[keys][0] += len(check_dispo)
                                tasks[keys][1] += len(check_dispo)
                    else : 
                        ok = True
                        ## Suppose that all the previous edges are ok
                        ## Update the ressources of the current edge
                        ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo
                ## Add info of a task
                tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,eva_rate,due_date])
        
                start += length
            current = nxt
            
#    print('tasks = ', tasks)
#    print('Nb of tasks = ',len(tasks))
    
    end_time = np.max([tasks[keys][1] for keys in tasks])
    solution = create_solution(tasks,list_eva_nodes)
 
    return end_time,solution


In [178]:
def get_end_time_2(LIST_EVA_NODES,EVA_TREE,GRAPH) : 
    ressources = {}
    for edge in GRAPH :
        edge_cap = edge[-1]
#         print('max cap of edge [{}-{}] : {}'.format(edge[0],edge[1],edge_cap))
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(1000,edge_cap))
#         print(ressources)
   
    tasks = {}
    for i in LIST_EVA_NODES : 
        nb_evacuees,max_rate,route_length,route_list = get_eva_node_info(i,EVA_TREE)
        start = 0
        duration,demande_res,eva_rate = get_task(i,EVA_TREE,GRAPH)
        current = i
#        print(eva_rate)
        for j in demande_res : 
#             print('Evacuees from {} at node {}'.format(i,j))
            nxt = j
            if current != nxt :
                due_date,length,edge_cap = get_edge_info(current,nxt,GRAPH)
                
                #Second solution variable 
                start2= start
                error_rate = 0
                
                
                cumul_time = 0
                capacity_ok = False
                
                #check the capicity constraints
                while (not capacity_ok) :
                    #check the avalable ressource
                    if current < nxt :
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                    else : 
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(nxt,current)]))
                    dispo[start+cumul_time:start+cumul_time+duration] -= eva_rate
#                     print('dispo[{}-{}]={}'.format(current,nxt,dispo))
                    check_dispo = [item for item in dispo[start+cumul_time:start+cumul_time+duration] if item < 0]
#                     index = [i for i in range(1000) if dispo[i]<0]
#                     print(index)
                    
                    if (len(check_dispo) > 0) :  #if OVERLOAD, shift the task
                        capacity_ok = False
                        cumul_time += len(check_dispo)
                        error_rate = -check_dispo[0]
#                         print('OVERLOAD',start+cumul_time+duration)
                    else : # move to the next task
                        capacity_ok = True
                
                #here there is no more conflict fot the current task
                #now compare the solutions
                
                
                #First solution : start+cumul_time,end=start +cumul_time + duration, rate = eva_rate
                #Second solution : 
                eva_rate2 = eva_rate - error_rate
                end_time1 = start + cumul_time + duration
                
                if (eva_rate2 > 0 and start2 + math.ceil(nb_evacuees / eva_rate2) < end_time1) :
#                     print("choose 2")
                    #if solutio 2 is better
                    #no change time start of the previous nodes
                    #decrease the rate
                    start = start2
                    eva_rate = eva_rate2
                    duration = math.ceil(nb_evacuees / eva_rate2)
                    if current < nxt :
                        dispo2 = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                    else : 
                        dispo2 = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(nxt,current)]))
                    dispo2[start:start+duration] -= eva_rate
                    # Change the eva_rate of the previous nodes 
                    for keys in tasks :
                        if 'Evacuees from {}'.format(i) in keys :
                            tasks[keys][3] = eva_rate
                            tasks[keys][2] = duration 
                    #update ressource
                    ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo2        
                else :        
#                     print("choose 1")
                    #if solution 1 is better
                    # Change the time start of the current nodes 
                    start += cumul_time
                    for keys in tasks :
                        if 'Evacuees from {}'.format(i) in keys :
                            tasks[keys][0] += cumul_time
                            tasks[keys][1] += len(check_dispo)
                    #update ressource
                    ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo
                
                tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,eva_rate])
        
                start += length
            
            current = nxt
            
#         print('ressources info after evacuation of node {} with rate{} = {}'.format(i,max_rate,ressources))
        
#    print('tasks = ', tasks)
#    print('Nb of tasks = ',len(tasks))
    end_time = np.max([tasks[keys][1] for keys in tasks])
#     print([tasks[keys][1] for keys in tasks])
#    print('End time is : ',end_time)
    solution = create_solution(tasks,LIST_EVA_NODES)
    
    return end_time,solution


random.shuffle(LIST_EVA_NODES)

end,sol = get_end_time(LIST_EVA_NODES,EVA_TREE,GRAPH)
print(end,sol)
end,sol = get_end_time_2(LIST_EVA_NODES,EVA_TREE,GRAPH)
print(end,sol)



193 [[344, 71, 0], [275, 70, 0], [163, 70, 25], [359, 70, 0], [323, 71, 78], [384, 71, 29], [285, 71, 74], [227, 71, 96], [279, 71, 102], [268, 132, 94]]
202 [[344, 71, 0], [275, 70, 0], [163, 70, 25], [359, 70, 0], [323, 71, 78], [384, 62, 29], [285, 67, 80], [227, 71, 103], [279, 67, 109], [268, 63, 0]]


In [ ]:
def get_latest_starttime(node,EVA_TREE,GRAPH):
    rate = get_task(node,EVA_TREE,GRAPH,None)[2]
    nb_evacuees,_,_,route_list = get_eva_node_info(node,EVA_TREE)
    route_list.reverse()
    route_list.append(node)
    #print(route_list)
    res, _, _ = get_edge_info(route_list[0],route_list[1],GRAPH)
    for i in range(1, len(route_list)-1):
        due_date, length, _ = get_edge_info(route_list[i],route_list[i+1],GRAPH)
        #print('Im on arc {}-{}'.format(route_list[i],route_list[i+1]))
        #print('R-L', res - length)
        #print('DD', due_date)
        if res-length > due_date:
            res = due_date
        else:
            res = res - length
        #print('R', res)
    return res - int(nb_evacuees/rate)

def get_list_priority(eva_tree,GRAPH) : 
    list_eva_nodes = [item[0] for item in eva_tree]
    maxstart = [(get_latest_starttime(item,eva_tree,GRAPH),item) for item in list_eva_nodes]
    print(maxstart)
    #sorted(maxstart, key= lambda item : item[1])
    maxstart.sort()
    print(maxstart)
    result = [item[1] for item in maxstart]
    return result
    
def get_end_time_3(LIST_EVA_NODES,EVA_TREE,GRAPH) : 
    ressources = {}
    for edge in GRAPH :
        edge_cap = edge[-1]
#         print('max cap of edge [{}-{}] : {}'.format(edge[0],edge[1],edge_cap))
        ressources.setdefault('Cap of edge[{}-{}]'.format(edge[0],edge[1]),np.full(500,edge_cap))
#         print(ressources)
   
    tasks = {}
    for i in LIST_EVA_NODES : 
        nb_evacuees,max_rate,route_length,route_list = get_eva_node_info(i,EVA_TREE)
        start = 0
        duration,demande_res,eva_rate = get_task(i,EVA_TREE,GRAPH)
        max_start = get_latest_starttime(i,EVA_TREE,GRAPH)
        current = i
#        print(eva_rate)
        #print('-------------Start at {} -------------'.format(i))

        for j in demande_res : 
#             print('Evacuees from {} at node {}'.format(i,j))
            nxt = j
            if current != nxt :
                due_date,length,edge_cap = get_edge_info(current,nxt,GRAPH)
                
                error_rate = 0
                cumul_time = 0
                capacity_ok = False
                
                #check the capicity constraints
                while (not capacity_ok) :
                    #check the avalable ressource
                    if current < nxt :
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                    else : 
                        dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(nxt,current)]))
                    dispo[start+cumul_time:start+cumul_time+duration] -= eva_rate
#                     print('dispo[{}-{}]={}'.format(current,nxt,dispo))
                    check_dispo = [item for item in dispo[start+cumul_time:start+cumul_time+duration] if item < 0]
#                     index = [i for i in range(1000) if dispo[i]<0]
#                     print(index)
                    
                    if (len(check_dispo) > 0) :  #if OVERLOAD, shift the task
                        capacity_ok = False
                        cumul_time += len(check_dispo)
                        error_rate = -np.max(check_dispo)
#                         print('OVERLOAD',start+cumul_time+duration)
                    else : # move to the next task
                        capacity_ok = True
                
                #here there is no more conflict fot the current task
                #now check the due date error
                duedate_error = False
                for key in tasks :
                    if (not duedate_error) :
                        if 'Evacuees from {}'.format(i) in key  :
                            if tasks[key][0] + tasks[key][2] + cumul_time > tasks[key][3] : 
                                duedate_error = True
                
                if duedate_error : #Change to second solution
                    #Second solution : 
                    eva_rate -= error_rate
                    if (eva_rate == 0) :
                        end = 9999
                    else :
                        duration = math.ceil(nb_evacuees / eva_rate)
                        # check again the capacity constraints
                        capacity_ok = False
                        while (not capacity_ok) :
                            #check the avalable ressource
                            if current < nxt :
                                dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(current,nxt)]))
                            else : 
                                dispo = np.copy(np.array(ressources['Cap of edge[{}-{}]'.format(nxt,current)]))
                            dispo[start+cumul_time:start+cumul_time+duration] -= eva_rate
        #                     print('dispo[{}-{}]={}'.format(current,nxt,dispo))
                            check_dispo = [item for item in dispo[start+cumul_time:start+cumul_time+duration] if item < 0]
        #                     index = [i for i in range(1000) if dispo[i]<0]
        #                     print(index)
                    
                            if (len(check_dispo) > 0) :  #if OVERLOAD, shift the task
                                capacity_ok = False
                                cumul_time += len(check_dispo)
                                error_rate = -np.max(check_dispo)
        #                         print('OVERLOAD',start+cumul_time+duration)
                            else : # move to the next task
                                capacity_ok = True
                        # Change the time start of the current nodes 
                        start += cumul_time
                        for keys in tasks :
                            if 'Evacuees from {}'.format(i) in keys :
                                tasks[keys][0] += cumul_time
                                tasks[keys][1] += len(check_dispo)
                        #update ressource
                        ressources['Cap of edge[{}-{}]'.format(current,nxt)] = dispo
                
                tasks.setdefault('Evacuees from {} at edge [{}-{}]'.format(i,current,nxt), [start,start+length+duration,duration,eva_rate])
        
                start += length
            current = nxt
            
#         print('ressources info after evacuation of node {} with rate{} = {}'.format(i,max_rate,ressources))
        
    #print('tasks = ', tasks)
    #print('Nb of tasks = ',len(tasks))
    
    
    for key in tasks :
        if (tasks[key][0] + tasks[key][2] > tasks[key][-1]) : 
            #print('Due Date violated !')
            tasks[keys][1] = 99999
#     print([tasks[keys][1] for keys in tasks])
#    print('End time is : ',end_time)
    end_time = np.max([tasks[keys][1] for keys in tasks])
    solution = create_solution(tasks,LIST_EVA_NODES)
    
        
#     for i  in LIST_EVA_NODES :
#         for key in tasks : 
#             if  ('Evacuees from {} at edge [{}-'.format(i,i) in key or ('Evacuees from {} at edge'.format(i) in key and '-{}]'.format(i) in key)) :
#                 print(i,key, tasks[key])
        
    return end_time,solution

random.shuffle(LIST_EVA_NODES)

end,sol = get_end_time(LIST_EVA_NODES,EVA_TREE,GRAPH)
print(end,sol)
end,sol = get_end_time_3(LIST_EVA_NODES,EVA_TREE,GRAPH)
print(end,sol)

In [3]:


#Run a local search to find a solution
endtime,best_solution,algo,exc = LocalSearchRandomStart2(EVA_TREE,GRAPH,n_iter=10,n_start_points=1)
create_solution_file3('dense_10_30_3_3_I',best_solution,endtime,algo,exc)

print('solution = ',best_solution)
print('end time = ',endtime)

# verify this solution by our checker
solution_path = os.path.dirname(os.path.abspath('__file__')) + "/Solutions/dense_10_30_3_1_I.solution"
verify_solution(data_path,solution_path)

print(check_a_solution(data_path,best_solution))
s = create_ordered_list_of(best_solution)
end,sol = get_end_time_2(s,EVA_TREE,GRAPH)
print(end,sol)
print(check_a_solution(data_path,sol))

---------------------------Start n.1---------------------------
Iteration 0:
[237, 447, 29, 138, 472, 17, 201, 505, 281, 382]  =>  421
Iteration 1:
[237, 447, 29, 138, 472, 17, 201, 505, 281, 382]  =>  421
Iteration 2:
[237, 447, 29, 138, 472, 17, 201, 505, 281, 382]  =>  421
Iteration 3:
[237, 447, 29, 138, 472, 17, 201, 505, 281, 382]  =>  421
Iteration 4:
[237, 447, 29, 138, 472, 17, 201, 505, 281, 382]  =>  421
----------------------------------------------------------------
solution =  [[237, 72, 0], [447, 70, 14], [29, 71, 35], [138, 70, 103], [472, 70, 121], [17, 70, 167], [201, 70, 222], [505, 70, 260], [281, 70, 298], [382, 70, 351]]
end time =  421
CREATION TASKS & VERIFY CONSTRAINTS
ERROR ON DUE DATE [138-449]
False
okk
262 [[237, 72, 0], [447, 66, 0], [29, 66, 21], [138, 70, 28], [472, 68, 46], [17, 70, 86], [201, 68, 120], [505, 62, 158], [281, 70, 154], [382, 68, 191]]
CREATION TASKS & VERIFY CONSTRAINTS
TOO MANY PERSONS AT EDGE [92-179] !!!
ERROR ON EDGE CAPACITY
False


In [3]:
# s = get_list_priority(EVA_TREE,GRAPH)
# print(s)
# #get_end_time_3(s,EVA_TREE,GRAPH)
# #[get_end_time_3(item,EVA_TREE,GRAPH)[0] for item in get_neighbors_of(s)]
# for item in get_neighbors_of(s) : 
#     e2,s2 = get_end_time_3(item,EVA_TREE,GRAPH)
#     print('------',e2)
#     x = []
#     for item2 in get_neighbors_of(create_ordered_list_of(s2)) :
#         e3,s3 = get_end_time_3(item2,EVA_TREE,GRAPH)
#         print('-----------------',e3)

In [11]:
# data_path = os.path.dirname(os.path.abspath('__file__')) + "/InstancesInt/dense_10_30_3_2_I.full"
# EVA_TREE,GRAPH,NB_NODES = read_data(data_path)
# NB_EVA_NODES = len(EVA_TREE)
# NB_EDGES = len(GRAPH)
# LIST_EVA_NODES = [item[0] for item in EVA_TREE]

# #Run a local search to find a solution

# endtime,best_solution,algo,exc = LocalSearchRandomStart3(EVA_TREE,GRAPH,n_iter=10,n_start_points=1)
# create_solution_file3('dense_10_30_3_1_I',best_solution,endtime,algo,exc)

# print('solution = ',best_solution)
# print('end time = ',endtime)

# # verify this solution by our checker
# solution_path = os.path.dirname(os.path.abspath('__file__')) + "/Solutions/dense_10_30_3_2_I.solution"
# verify_solution(data_path,solution_path)

---------------------------Start n.1---------------------------
Iteration 0:
[514, 277, 21, 284, 197, 501, 18, 356, 516, 431]  =>  99999
Iteration 1:
[514, 277, 21, 284, 197, 501, 18, 356, 516, 431]  =>  99999
Iteration 2:
[514, 277, 21, 284, 197, 501, 18, 356, 516, 431]  =>  99999
Iteration 3:
[514, 277, 21, 284, 197, 501, 18, 356, 516, 431]  =>  99999
Iteration 4:
[514, 277, 21, 284, 197, 501, 18, 356, 516, 431]  =>  99999
----------------------------------------------------------------
solution =  [[514, 192, 0], [277, 189, 0], [21, 126, 8], [501, 192, 41], [284, 165, 10], [18, 192, 86], [197, 183, 34], [356, 144, 123], [516, 192, 146], [431, 150, 209]]
end time =  99999
CREATION TASKS & VERIFY CONSTRAINTS
197 102
356 70
501 151
ERROR ON DUE DATE [0-501]


False